In [ ]:
!pip install seaborn imutils --quiet

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import math, random
import imutils
import os
import gc
from tqdm import tqdm
import tensorflow as tf
from tensorflow.train import BytesList, Int64List, Feature, Features, Example
from kaggle_datasets import KaggleDatasets
from kaggle_secrets import UserSecretsClient


In [ ]:
frame_cnt = 15
frame_width = 50
DEBUG = True

In [ ]:
TPU_MODE = True

if TPU_MODE:
    try: # detect TPUs
        # NEW: in Tensorflow 2.4
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        # instantiate a distribution strategy
        tf.tpu.experimental.initialize_tpu_system(tpu)
        tpu_strategy = tf.distribute.TPUStrategy(tpu)
    except ValueError: # otherwise detect GPUs
        strategy = tf.distribute.MirroredStrategy() # single-GPU or multi-GPU
    
    N_REPLICAS = tpu_strategy.num_replicas_in_sync
    
    print(f'Running on TPU {tpu.master()}') 
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Device is:',device)

# Training and validation data generator

In [ ]:
# Install this to prevent: ModuleNotFoundError: No module named 'tensorflow_gcs_config'
!pip install tensorflow-gcs-config

In [ ]:
# Need this part to access my own private dataset
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

# A description of the features.
features_dict = {
    'image': tf.io.FixedLenFeature([frame_width*frame_width*frame_cnt], tf.float32),
    'label': tf.io.FixedLenFeature([2], tf.int64),
  }

if TPU_MODE:
    BATCH_SIZE = 16 * N_REPLICAS
else:
    BATCH_SIZE = 32

print('BATCH SIZE:', BATCH_SIZE)

def parse_tfrecord(example,visualize = False):
    sample_record = tf.io.parse_single_example(example,features_dict)
    feats = sample_record['image']
    label = sample_record['label']
    
    feats = tf.reshape(feats,[frame_cnt,frame_width,frame_width])
    feats = tf.transpose(feats, [1,2,0]) # in Conv3D we need (h,w,depth)
    label = tf.reshape(label,[2])

    return feats,label

GCS_PATH = KaggleDatasets().get_gcs_path('bsms155050')
if DEBUG:
    print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
    for file in tf.io.gfile.glob(os.path.join(GCS_PATH, "*")): print(f"\t--> {file}")

tf_pos_records = tf.io.gfile.glob(GCS_PATH + '/pos/*.tfrecord')
tf_neg_records = tf.io.gfile.glob(GCS_PATH + '/neg/*.tfrecord')

AUTO = tf.data.experimental.AUTOTUNE
tf_pos_dataset = tf.data.TFRecordDataset(tf_pos_records, num_parallel_reads=AUTO).map(parse_tfrecord)
tf_neg_dataset = tf.data.TFRecordDataset(tf_neg_records, num_parallel_reads=AUTO).map(parse_tfrecord)

tf_pos_trn_dataset = tf_pos_dataset.skip(512)
tf_pos_val_dataset = tf_pos_dataset.take(512)

tf_neg_trn_dataset = tf_neg_dataset.skip(512)
tf_neg_val_dataset = tf_neg_dataset.take(512)

tf_pos_trn_dataset = tf_pos_trn_dataset.repeat(300).shuffle(BATCH_SIZE*10)
tf_neg_trn_dataset = tf_neg_trn_dataset.repeat(90).shuffle(BATCH_SIZE*10)

#combined_dataset = tf_pos_dataset.concatenate(tf_neg_dataset).shuffle(BATCH_SIZE*10)
combined_trn_dataset = tf.data.Dataset.sample_from_datasets([tf_pos_trn_dataset, tf_neg_trn_dataset], weights=[0.5, 0.5]).batch(BATCH_SIZE)
combined_val_dataset = tf.data.Dataset.sample_from_datasets([tf_pos_val_dataset, tf_neg_val_dataset], weights=[0.5, 0.5]).batch(4)

# print('The pos dataset size:',len(list(tf_pos_dataset)))
# print('The neg dataset size:',len(list(tf_neg_dataset)))
# print('The final dataset size:',len(list(combined_dataset)))

In [ ]:
# Validation - let's check if we recorded images and labels correctly.

# Read from file and show
filenames = ['/kaggle/input/bsms155050/pos/100147863_0.tfrecord']
raw_dataset = tf.data.TFRecordDataset(filenames)

def visualize_images(img_arr,frame_cnt):
    a = 5
    b = frame_cnt // a

    fig, axarr = plt.subplots(b, a, constrained_layout = True, figsize=(5, 5))

    for i in range(b):
        for j in range(a):
            axarr[i, j].set_xticklabels([])
            axarr[i, j].set_yticklabels([])
            axarr[i, j].imshow(img_arr[:,:,i*a+j], cmap='gray')
    plt.show()
    

parsed_dataset = raw_dataset.map(parse_tfrecord)
for img, label in parsed_dataset:
    print(img.shape)
    print(label)
    visualize_images(img,frame_cnt)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import activations

class BC3DConvNet(tf.keras.Model):
    def __init__(self, frame_cnt, frame_width):
        super(BC3DConvNet, self).__init__()

        self.conv1 = Conv2D(filters=frame_cnt*16, kernel_size=(3, 3), input_shape=(frame_width, frame_width, frame_cnt))
        self.bn1 = BatchNormalization()
        self.conv2 = Conv2D(filters=frame_cnt*16, kernel_size=(3, 3))
        self.bn2 = BatchNormalization()
        self.conv3 = Conv2D(filters=frame_cnt*32, kernel_size=(3, 3))
        self.bn3 = BatchNormalization()
        self.conv4 = Conv2D(filters=frame_cnt*64, kernel_size=(3, 3))
        self.bn4 = BatchNormalization()

        # Calculate the convolution's output size:
        tmp = np.zeros((1, frame_width, frame_width, frame_cnt))
        x = self.apply_conv(tmp)
        x = Flatten()(x)
        linear_input = x.shape[1]
        # ------------------------------------------------------

        self.fc1 = Dense(512, activation='relu')
        self.bn5 = BatchNormalization()
        self.fc2 = Dense(1024, activation='relu')
        self.bn6 = BatchNormalization()
        self.fc3 = Dense(512, activation='relu')
        self.bn7 = BatchNormalization()
        self.fc4 = Dense(2, activation='softmax')

    def apply_conv(self, x):
        x = activations.relu(self.conv1(x))
        x = self.bn1(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        x = activations.relu(self.conv2(x))
        x = self.bn2(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        x = activations.relu(self.conv3(x))
        x = self.bn3(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        x = activations.relu(self.conv4(x))
        x = self.bn4(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        return x

    def call(self, x):
        conv_out = self.apply_conv(x)
        x = Flatten()(conv_out)
        x = self.fc1(x)
        x = self.bn5(x)
        x = Dropout(0.4)(x)
        x = self.fc2(x)
        x = self.bn6(x)
        x = Dropout(0.4)(x)
        x = self.fc3(x)
        x = self.bn7(x)
        x = Dropout(0.4)(x)
        x = self.fc4(x)
        return x


In [ ]:
import keras

EPOCHS = 200
STEPS = 50

with tpu_strategy.scope():
    net = BC3DConvNet(frame_cnt, frame_width)
    net.compile(loss='binary_crossentropy', 
                optimizer=keras.optimizers.Adam(learning_rate=0.0005,weight_decay=0.15,clipnorm=1.0), 
                steps_per_execution=32,
                metrics=['accuracy'])

In [ ]:
history = net.fit(combined_trn_dataset, epochs=EPOCHS, steps_per_epoch=STEPS, validation_data = combined_val_dataset, validation_steps=4)

In [ ]:
net.summary()

In [ ]:
import matplotlib.pyplot as plt

# Plotting training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plotting training and validation loss
plt.plot(history.history['loss'][50:], label='Training Loss')
plt.plot(history.history['val_loss'][50:], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plotting training and validation loss
plt.plot(history.history['loss'][:50], label='Training Loss')
plt.plot(history.history['val_loss'][:50], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
net.save_weights('BC3DConvNet', save_format='tf')